# Abstract
Predictive coding offers a theoretical framework for understanding how the brain processes novel stimuli and continuously updates its internal representations of the world. This framework elucidates how the brain minimizes redundancy and encodes information efficiently. Empirical research has highlighted variations in the preference of ‘familiarity/novelty-encoding neurons’ across different cortical depths. Despite these insights, the specific localized encoding mechanisms crucial for explaining functional connectivity in decision-making tasks remain unknown.
We hypothesize that vasoactive intestinal peptide (VIP) interneurons in the deeper cortical layers play a significant role in modulating predictive coding. Specifically, we propose that these VIP interneurons suppress somatostatin (SST) inhibitory feedback, which targets novelty-encoding excitatory neurons in the shallower cortical layers. This suppression is expected to maintain the activity of excitatory neurons involved in encoding novel stimuli, thereby facilitating their contribution to the generation and updating of predictive models in response to new information.
To test this hypothesis, we will utilize the Allen Institute dataset, which includes 2-photon calcium imaging of neural activity in the visual cortex of mice engaged in behavioral tasks involving the detection of novel and familiar images. Our approach involves performing a time-series analysis to assess the impact of VIP activation on novelty-encoding excitatory populations in superficial layers. We will employ machine learning classifiers to analyze neuron activation from calcium fluorescence signals. The effects will be measured by evaluating delays in neuron activation and examining the dynamics of the VIP-SST disinhibitory circuit on excitatory populations.
We anticipate observing activation of VIP interneurons in the absence of activity from familiarity-encoding excitatory populations in deeper layers, followed by subsequent activation of novelty-encoding excitatory neurons in shallower layers. Such findings would suggest a feedback mechanism modulating predictive coding functions in cortical regions near the retina.

1. Function to average neuron dff values over time def avg_dff_vals(list_of_neurons, time_interval) -> ndarray containing avg dff values **with the timestep at each avg**
  a. Plot it
2. ML classifier to identify activations from dff data (over time). [0.002, 0.005, 0.2, 0.4, 0.8, 0.85, 0.83, 0.4, 0.09, 0.01] -> t=[6]
3. Find how to link imaging for VIP, SST, and excitatory cells in a single task session


Future
1. find novelty/familiarity-encoding excitatory neurons
2. plot PSTH of E/V/P avg activity per layer

# Setup

In [ ]:
!python -m pip install pip --upgrade --quiet
!python -m pip install pandas --quiet
!python -m pip install seaborn --quiet
!python -m pip install numpy scipy matplotlib ipython jupyter pandas sympy nose --quiet
!python -m pip install allensdk
!python -m pip install brain_observatory_utilities --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 17.2 MB/s eta 0:00:

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

import brain_observatory_utilities.datasets.optical_physiology.data_formatting as ophys_formatting
import brain_observatory_utilities.utilities.general_utilities as utilities

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)
# this line may be needed if you run into Error in pandas query function
# Otherwise set the engine to python in queries made throught the book
# pd.DataFrame.query = lambda self, expr, **kwargs: self.query(expr, engine='python', **kwargs)

* Cross validation on excitatory and inhibitory
* PSTH for every neuron and average them over all the neurons to show excitatory are going up
* PCA to reduce / visualize dimensionality of the data - can filter down by layer, task (novel or familiar), or excitatory/SST/VIP

In [ ]:
data_storage_directory = "./temp"  # Note: this path must exist on your local drive
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()

ophys_session_table.csv: 100%|██████████| 247k/247k [00:00<00:00, 1.94MMB/s] 
behavior_session_table.csv: 100%|██████████| 1.59M/1.59M [00:00<00:00, 6.89MMB/s]
ophys_experiment_table.csv: 100%|██████████| 657k/657k [00:00<00:00, 4.10MMB/s] 
ophys_cells_table.csv: 100%|██████████| 4.28M/4.28M [00:00<00:00, 6.89MMB/s]
/usr/local/lib/python3.10/dist-packages/allensdk/brain_observatory/behavior/behavior_project_cache/behavior_project_cache.py:135: UpdatedStimulusPresentationTableWarning: 
	As of AllenSDK version 2.16.0, the latest Visual Behavior Ophys data has been significantly updated from previous releases. Specifically the user will need to update all processing of the stimulus_presentations tables. These tables now include multiple stimulus types delineated by the columns `stimulus_block` and `stimulus_block_name`.

The data that was available in previous releases are stored in the block name containing 'change_detection' and can be accessed in the pandas table by using: 
	`stimulus_

In [ ]:
print(session_table['cre_line'].unique())
session_table.head()

['Sst-IRES-Cre' 'Vip-IRES-Cre' 'Slc17a7-IRES2-Cre']


,behavior_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,age_in_days,imaging_plane_group_count,project_code,session_type,session_number,image_set,behavior_type,experience_level,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,date_of_acquisition,equipment_name,num_depths_per_area,ophys_experiment_id,num_targeted_structures
ophys_session_id,,,,,,,,,,,,,,,,,,,,,,,,,
951410079,951520319,"[1018028339, 1018028342, 1018028345, 101802835...",457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,4,VisualBehaviorMultiscope,OPHYS_1_images_A,1,images_A,active_behavior,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,4,"[951980471, 951980473, 951980475, 951980479, 9...",2
952430817,952554548,"[1018028339, 1018028345, 1018028354, 1018028357]",457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,209,3,VisualBehaviorMultiscope,OPHYS_2_images_A_passive,2,images_A,passive_viewing,Familiar,0,66,1,2019-09-23 08:45:38.490000+00:00,MESO.1,4,"[953659743, 953659745, 953659749, 953659752]",2
954954402,953982960,"[1018028339, 1018028342, 1018028345, 101802835...",457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,210,4,VisualBehaviorMultiscope,OPHYS_3_images_A,3,images_A,active_behavior,Familiar,0,67,2,2019-09-24 09:01:31.582000+00:00,MESO.1,4,"[958527464, 958527471, 958527474, 958527479, 9...",2
955775716,956010809,"[1018028339, 1018028342, 1018028345]",457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,212,2,VisualBehaviorMultiscope,OPHYS_3_images_A,3,images_A,active_behavior,Familiar,1,68,3,2019-09-26 09:22:21.772000+00:00,MESO.1,4,"[956941841, 956941844, 956941846]",2
957020350,957032492,"[1018028339, 1018028342, 1018028345, 101802835...",457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,213,4,VisualBehaviorMultiscope,OPHYS_4_images_B,4,images_B,active_behavior,Novel 1,0,0,4,2019-09-27 08:58:37.005000+00:00,MESO.1,4,"[957759562, 957759564, 957759566, 957759570, 9...",2


In [ ]:
experiment_table.head()

,behavior_session_id,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,age_in_days,imaging_depth,targeted_structure,targeted_imaging_depth,imaging_plane_group,project_code,session_type,session_number,image_set,behavior_type,passive,experience_level,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,date_of_acquisition,equipment_name,published_at,isi_experiment_id,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951980471,951520319,951410079,1018028342,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISp,150,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,0
951980473,951520319,951410079,1018028345,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISp,225,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,1
951980475,951520319,951410079,1018028339,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,75,VISp,75,1,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,2
951980479,951520319,951410079,1018028354,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISl,150,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,3
951980481,951520319,951410079,1018028357,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISl,225,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,4


In [ ]:
def get_experiments(ophys_experiment_ids):
  experiments = {}
  # for ophys_session_id in ophys_session_ids:
  #   ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
  for ophys_experiment_id in ophys_experiment_ids:
      experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)
  return experiments

def get_neural_data(experiments):
  neural_data = []
  for ophys_experiment_id in tqdm(experiments.keys()): #tqdm is a package that shows progress bars for items that are iterated over
      this_experiment = experiments[ophys_experiment_id]
      this_experiment_neural_data = ophys_formatting.build_tidy_cell_df(this_experiment)

      # add some columns with metadata for the experiment
      metadata_keys = [
        'ophys_experiment_id',
        'ophys_session_id',
        'targeted_structure',
        'imaging_depth',
        'equipment_name',
        'cre_line',
        'mouse_id',
        'sex',
      ]
      for metadata_key in metadata_keys:
          this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

      # append the data for this experiment to a list
      neural_data.append(this_experiment_neural_data)

  # concatate the list of dataframes into a single dataframe
  neural_data = pd.concat(neural_data)
  return neural_data

def avg_dff_vals(neural_data: pd.DataFrame, time_interval: float):
    """
    Averages neuron dF/F values over specified time intervals.
    # Example usage:
    # Assuming 'neural_data' is your DataFrame and you want 100 timestamps
    num_timestamps = 100
    total_time = neural_data['timestamps'].max() - neural_data['timestamps'].min()
    time_interval = total_time / num_timestamps
    cell_ids_example = [1086613265]
    avg_dff = avg_dff_vals(neural_data[neural_data['cell_specimen_id'].isin(cell_ids_example)], time_interval)
    print(avg_dff)

    Args:
        neural_data: A pandas DataFrame containing neural data.
        time_interval: The time interval over which to average the dF/F values.

    Returns:
        A numpy ndarray containing the average dF/F values, with the timestep at each average.
    """
    # Create a new column for time bins based on the specified time interval
    neural_data_copy = neural_data.copy()
    neural_data_copy['time_bin'] = (neural_data['timestamps'] // time_interval).astype(int) * time_interval
    # Group by cell_specimen_id and time_bin, then calculate the mean dF/F for each group
    grouped_data = neural_data_copy.groupby(['cell_specimen_id', 'time_bin'])['dff'].mean().reset_index()
    # Pivot the table to have cell IDs as columns and time bins as index
    pivoted_data = grouped_data.pivot(index='time_bin', columns='cell_specimen_id', values='dff')
    pivoted_data['pop_avg'] = pivoted_data.mean(axis=1)
    # Rename index for clarity
    pivoted_data.index.name = 'timestep'

    return pivoted_data


def plot_avg_dff(avg_dff: pd.DataFrame, cell_ids: list):
  """ Plot the average dF/F values over time for the specified cells.
  Args:
      avg_dff: A pandas DataFrame containing the average dF/F values, with the timestep at each average.
      cell_ids: A list of cell IDs for which to plot the average dF/F values.
  """
  # Plot the average dF/F values over time for the specified cells
  for cell_id in cell_ids:
    if cell_id in avg_dff.columns:
        cell_data = avg_dff[[cell_id]]
        plt.plot(cell_data.index, cell_data[cell_id], label=f'Cell {cell_id}')

  plt.xlabel('Timestep')
  plt.ylabel('Average dF/F')
  plt.title('Average dF/F Over Time')
  plt.legend()

In [ ]:
exp = get_experiments([951980471])
neural_data = get_neural_data(exp)

behavior_ophys_experiment_951980471.nwb: 100%|██████████| 248M/248M [00:11<00:00, 21.6MMB/s]
/usr/local/lib/python3.10/dist-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


In [ ]:
# Example usage:
# Assuming 'neural_data' is your DataFrame and you want 100 timestamps
num_timestamps = 100
total_time = neural_data['timestamps'].max() - neural_data['timestamps'].min()
time_interval = 1
cell_ids_example = [1086613265]
avg_dff = avg_dff_vals(neural_data, time_interval)
print(avg_dff)

cell_specimen_id  1086613265  1086613823  1086619526  1086614149  1086614351  \
timestep                                                                       
9                   1.124236    0.294942    0.162461    0.123916    0.171194   
10                  1.174628    0.188839    0.028613    0.028337    0.022779   
11                  1.729501    0.130319    0.040770    0.037850    0.019069   
12                  1.357256    0.118184   -0.001823   -0.047289    0.016254   
13                  0.932193    0.119745    0.003426    0.044430    0.027713   
...                      ...         ...         ...         ...         ...   
4509               -0.029174   -0.003984   -0.025576   -0.021253    0.006155   
4510                0.000500   -0.052018    0.057029    0.035463    0.034391   
4511                1.174392   -0.029650    0.020653   -0.022807   -0.028396   
4512                0.250718    0.543355   -0.031437    0.090584    0.048612   
4513                0.026897    0.752867

In [ ]:
visp_data = neural_data.query('targeted_structure == "VISp"')
cell_ids = visp_data['cell_specimen_id'].unique()
print('there are {} unique cells'.format(len(cell_ids)))
print('cell ids are: {}'.format(cell_ids))
print(visp_data.head())



there are 12 unique cells
cell ids are: [1086613265, 1086613823, 1086619526, 1086614149, 1086614351, ..., 1086615620, 1086615837, 1086616206, 1086619674, 1086616398]
Length: 12
Categories (12, int64): [1086613265, 1086613823, 1086619526, 1086614149, ..., 1086615837, 1086616206,
                         1086619674, 1086616398]
   timestamps       dff    events  filtered_events cell_roi_id  \
0     9.26356  0.936573  0.000000         0.000000  1080743723   
1     9.35677  0.582486  0.000000         0.000000  1080743723   
2     9.44998  1.296005  0.556873         0.400762  1080743723   
3     9.54318  0.844898  0.000000         0.148502  1080743723   
4     9.63639  1.181188  0.467264         0.343829  1080743723   

  cell_specimen_id  ophys_experiment_id  ophys_session_id targeted_structure  \
0       1086613265            951980471         951410079               VISp   
1       1086613265            951980471         951410079               VISp   
2       1086613265            95198

In [ ]:
example_session_id = session_table.index[0]
print(example_session_id)
print(session_table.loc[example_session_id])

951410079
behavior_session_id                                                        951520319
ophys_container_id                 [1018028339, 1018028342, 1018028345, 101802835...
mouse_id                                                                      457841
indicator                                                                    GCaMP6f
full_genotype                          Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt
driver_line                                                           [Sst-IRES-Cre]
cre_line                                                                Sst-IRES-Cre
reporter_line                                             Ai148(TIT2L-GC6f-ICL-tTA2)
sex                                                                                F
age_in_days                                                                      206
imaging_plane_group_count                                                          4
project_code                                           

In [ ]:
visp_experiment_indexes = experiment_table.loc[experiment_table['ophys_session_id'] == example_session_id].query('targeted_structure == "VISp"').index
print(visp_experiment_indexes)

Int64Index([951980471, 951980473, 951980475], dtype='int64', name='ophys_experiment_id')


In [ ]:
experiment_table.loc[951980471].stimulus_presentations

AttributeError: 'Series' object has no attribute 'stimulus_presentations'

In [ ]:
experiments = get_experiments(visp_experiment_indexes.tolist())

In [ ]:
neural_data = get_neural_data(experiments)

In [ ]:
print(len(neural_data))
cell_ids = neural_data['cell_specimen_id'].unique()
print('there are {} unique cells'.format(len(cell_ids)))
print('cell ids are: {}'.format(cell_ids))
print(neural_data.head())
print(neural_data['imaging_depth'].unique())
print(neural_data['cre_line'].unique())
print(neural_data['mouse_id'].unique())

In [ ]:
one_visp = visp_data.query('cell_specimen_id == 1086613265')
print(one_visp)
one_visp['timestamps'][0:500]
plt.figure(figsize=(30, 6))
plt.plot(one_visp['timestamps'][0:1500], one_visp['dff'][0:1500])

Goal: Plot responses from excitatory, VIP, and SST during 1) familiar 2) novel